In [1]:
# IMPORTS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math 
import pylab 

from matplotlib.dates import DateFormatter
from glob import glob
from os import path
from collections import defaultdict
#from virocon import (
    #GlobalHierarchicalModel,
    #ExponentiatedWeibullDistribution,
    #WeibullDistribution,
    #DependenceFunction,
    #WidthOfIntervalSlicer,
    #plot_marginal_quantiles,
    #plot_dependence_functions,
#)

import sys
sys.path.insert(0, path.abspath(path.join(path.curdir, '../src')))

%matplotlib notebook

# Load all needed data

## Wind

In [2]:
# WIND DATA

#wind_data = pd.read_csv('/Volumes/MASTERTHESE/MasterThese/wind_data_resampled1s_UTC.csv')
wind_data = pd.read_csv('/Volumes/MASTERTHESE/MasterThese/wind_data_resampled1min_UTC.csv')
wind_data.set_index('datetime', inplace=True)
#wind_data.index= wind_data.index.tz_convert('UTC')
wind_data.index= pd.DatetimeIndex(wind_data.index)
print(type(wind_data.index[-1]))
print(wind_data.index[-1])

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2019-12-25 23:59:00+00:00


## Waves

In [3]:
# WAVE DATA

#wave_data = pd.read_csv('/Volumes/MASTERTHESE/MasterThese/wave_data_resampled1s_UTC.csv')
wave_data = pd.read_csv('/Volumes/MASTERTHESE/MasterThese/wave_data_resampled1min_UTC.csv')
wave_data.set_index('time', inplace=True)
wave_data.index= pd.DatetimeIndex(wave_data.index)
print(type(wave_data.index[-1]))
print(wave_data.index[-1])
print(wave_data)

/Applications/anaconda3/envs/env_masterthese/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2019-12-25 23:30:00+00:00
                            Zeitpunkt gerundet                      Date/Time  \
time                                                                            
2019-08-03 00:00:00+00:00  2019-08-03 00:00:00  2019-08-03T00:05:00.000+02:00   
2019-08-03 00:01:00+00:00  2019-08-03 00:00:00  2019-08-03T00:05:00.000+02:00   
2019-08-03 00:02:00+00:00  2019-08-03 00:00:00  2019-08-03T00:05:00.000+02:00   
2019-08-03 00:03:00+00:00  2019-08-03 00:00:00  2019-08-03T00:05:00.000+02:00   
2019-08-03 00:04:00+00:00  2019-08-03 00:00:00  2019-08-03T00:05:00.000+02:00   
...                                        ...                            ...   
2019-12-25 23:26:00+00:00  2019-12-25 23:00:00  2019-12-25T23:04:59,999+01:00   
2019-12-25 23:27:00+00:00  2019-12-25 23:00:00  2019-12-25T23:04:59,999+01:00   
2019-12-25 23:28:00+00:00  2019-12-25 23:00:00  2019-12-25T23:04:59,999+01:00   
2019-12-25 23:29:00+00:00  2019-

## Blade landings

In [4]:
# BLADE LANDINGS BAS

blade_landings= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/blade_landing_helihoist_sbitroot.txt')

# make epoch index of dataframe and convert epoch into readable dates
blade_landings.epoch_land= blade_landings.epoch_land.apply(lambda x: pd.to_datetime(x, unit= 's', utc=True))

# convert blade landings of turbine 8 into readable dates
#blade_landing1= pd.to_datetime(1.571127e+09, unit='s', utc=True)
#blade_landing2= pd.to_datetime(1.571154e+09, unit='s', utc=True)
#blade_landing3= pd.to_datetime(1.571188e+09, unit='s', utc=True)
#landings = (blade_landing1, blade_landing2, blade_landing3)

landings_tb4 = (pd.to_datetime(blade_landings.epoch_land[0]))
landings_tb5 = (pd.to_datetime(blade_landings.epoch_land[1]), pd.to_datetime(blade_landings.epoch_land[2]), pd.to_datetime(blade_landings.epoch_land[3]))
landings_tb6 = (pd.to_datetime(blade_landings.epoch_land[4]), pd.to_datetime(blade_landings.epoch_land[5]))
landings_tb7 = (pd.to_datetime(blade_landings.epoch_land[6]), pd.to_datetime(blade_landings.epoch_land[7]), pd.to_datetime(blade_landings.epoch_land[8]))
landings_tb8 = (pd.to_datetime(blade_landings.epoch_land[9]), pd.to_datetime(blade_landings.epoch_land[10]), pd.to_datetime(blade_landings.epoch_land[11]))
landings_tb10 = (pd.to_datetime(blade_landings.epoch_land[12]))
landings_tb11 = (pd.to_datetime(blade_landings.epoch_land[13]), pd.to_datetime(blade_landings.epoch_land[14]))
landings_tb12 = (pd.to_datetime(blade_landings.epoch_land[15]), pd.to_datetime(blade_landings.epoch_land[16]))
landings_tb13 = (pd.to_datetime(blade_landings.epoch_land[17]), pd.to_datetime(blade_landings.epoch_land[18]))
landings_tb14 = (pd.to_datetime(blade_landings.epoch_land[19]), pd.to_datetime(blade_landings.epoch_land[20]), pd.to_datetime(blade_landings.epoch_land[21]))

## GPS Data

In [5]:
# READ CSV files of GPS Data: turbine 4

# sbitroot
gps_sbitroot_tb4= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-04_sbitroot_tom_gps.csv')
gps_sbitroot_tb4.epoch= pd.to_datetime(gps_sbitroot_tb4.epoch, unit= 's', utc=True)
gps_sbitroot_tb4.set_index('epoch', inplace=True)

# sbittip
gps_sbittip_tb4= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-04_sbittip_tom_gps.csv')
gps_sbittip_tb4.epoch= pd.to_datetime(gps_sbittip_tb4.epoch, unit= 's', utc=True)
gps_sbittip_tb4.set_index('epoch', inplace=True)

# helihoist
gps_helih_tb4= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-04_helihoist-1_tom_gps.csv')
gps_helih_tb4.epoch= pd.to_datetime(gps_helih_tb4.epoch, unit= 's', utc=True)
gps_helih_tb4.set_index('epoch', inplace=True)

In [6]:
# READ CSV files of GPS Data: turbine 5

# sbitroot
gps_sbitroot_tb5= pd.read_csv('/Volumes/MASTERTHESE//MasterThese/GPS/turbine-05_sbitroot_tom_gps.csv')
gps_sbitroot_tb5.epoch= pd.to_datetime(gps_sbitroot_tb5.epoch, unit= 's', utc=True)
gps_sbitroot_tb5.set_index('epoch', inplace=True)

# sbittip
gps_sbittip_tb5= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-05_sbittip_tom_gps.csv')
gps_sbittip_tb5.epoch= pd.to_datetime(gps_sbittip_tb5.epoch, unit= 's', utc=True)
gps_sbittip_tb5.set_index('epoch', inplace=True)

# helihoist
gps_helih_tb5= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-05_helihoist-1_tom_gps.csv')
gps_helih_tb5.epoch= pd.to_datetime(gps_helih_tb5.epoch, unit= 's', utc=True)
gps_helih_tb5.set_index('epoch', inplace=True)

In [7]:
# READ CSV files of GPS Data: turbine 6

# sbitroot
gps_sbitroot_tb6= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-06_sbitroot_tom_gps.csv')
gps_sbitroot_tb6.epoch= pd.to_datetime(gps_sbitroot_tb6.epoch, unit= 's', utc=True)
gps_sbitroot_tb6.set_index('epoch', inplace=True)

# sbittip
gps_sbittip_tb6= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-06_sbittip_tom_gps.csv')
gps_sbittip_tb6.epoch= pd.to_datetime(gps_sbittip_tb6.epoch, unit= 's', utc=True)
gps_sbittip_tb6.set_index('epoch', inplace=True)

# helihoist
gps_helih_tb6= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-06_helihoist-1_tom_gps.csv')
gps_helih_tb6.epoch= pd.to_datetime(gps_helih_tb6.epoch, unit= 's', utc=True)
gps_helih_tb6.set_index('epoch', inplace=True)

In [8]:
# READ CSV files of GPS Data: turbine 7

# sbitroot
gps_sbitroot_tb7= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-07_sbitroot_tom_gps.csv')
gps_sbitroot_tb7.epoch= pd.to_datetime(gps_sbitroot_tb7.epoch, unit= 's', utc=True)
gps_sbitroot_tb7.set_index('epoch', inplace=True)

# sbittip
gps_sbittip_tb7= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-07_sbittip_tom_gps.csv')
gps_sbittip_tb7.epoch= pd.to_datetime(gps_sbittip_tb7.epoch, unit= 's', utc=True)
gps_sbittip_tb7.set_index('epoch', inplace=True)

# helihoist
gps_helih_tb7= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-07_helihoist-1_tom_gps.csv')
gps_helih_tb7.epoch= pd.to_datetime(gps_helih_tb7.epoch, unit= 's', utc=True)
gps_helih_tb7.set_index('epoch', inplace=True)

In [9]:
# READ CSV files of GPS Data: turbine 8

# sbitroot
gps_sbitroot_tb8= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-08_sbitroot_tom_gps.csv')
gps_sbitroot_tb8.epoch= pd.to_datetime(gps_sbitroot_tb8.epoch, unit= 's', utc=True)
gps_sbitroot_tb8.set_index('epoch', inplace=True)

# sbittip
gps_sbittip_tb8= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-08_sbittip_tom_gps.csv')
gps_sbittip_tb8.epoch= pd.to_datetime(gps_sbittip_tb8.epoch, unit= 's', utc=True)
gps_sbittip_tb8.set_index('epoch', inplace=True)

# helihoist
gps_helih_tb8= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-08_helihoist-1_tom_gps.csv')
gps_helih_tb8.epoch= pd.to_datetime(gps_helih_tb8.epoch, unit= 's', utc=True)
gps_helih_tb8.set_index('epoch', inplace=True)

In [10]:
# READ CSV files of GPS Data: turbine 10

# sbitroot
gps_sbitroot_tb10= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-10_sbitroot_tom_gps.csv')
gps_sbitroot_tb10.epoch= pd.to_datetime(gps_sbitroot_tb10.epoch, unit= 's', utc=True)
gps_sbitroot_tb10.set_index('epoch', inplace=True)

# sbittip
gps_sbittip_tb10= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-10_sbittip_tom_gps.csv')
gps_sbittip_tb10.epoch= pd.to_datetime(gps_sbittip_tb10.epoch, unit= 's', utc=True)
gps_sbittip_tb10.set_index('epoch', inplace=True)

# helihoist
gps_helih_tb10= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-10_helihoist-1_tom_gps.csv')
gps_helih_tb10.epoch= pd.to_datetime(gps_helih_tb10.epoch, unit= 's', utc=True)
gps_helih_tb10.set_index('epoch', inplace=True)

In [11]:
# READ CSV files of GPS Data: turbine 11

# sbitroot
gps_sbitroot_tb11= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-11_sbitroot_tom_gps.csv')
gps_sbitroot_tb11.epoch= pd.to_datetime(gps_sbitroot_tb11.epoch, unit= 's', utc=True)
gps_sbitroot_tb11.set_index('epoch', inplace=True)

# sbittip
gps_sbittip_tb11= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-11_sbittip_tom_gps.csv')
gps_sbittip_tb11.epoch= pd.to_datetime(gps_sbittip_tb11.epoch, unit= 's', utc=True)
gps_sbittip_tb11.set_index('epoch', inplace=True)

# helihoist
gps_helih_tb11= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-11_helihoist-1_tom_gps.csv')
gps_helih_tb11.epoch= pd.to_datetime(gps_helih_tb11.epoch, unit= 's', utc=True)
gps_helih_tb11.set_index('epoch', inplace=True)

In [12]:
# READ CSV files of GPS Data: turbine 12

# sbitroot
gps_sbitroot_tb12= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-12_sbitroot_tom_gps.csv')
gps_sbitroot_tb12.epoch= pd.to_datetime(gps_sbitroot_tb12.epoch, unit= 's', utc=True)
gps_sbitroot_tb12.set_index('epoch', inplace=True)

# sbittip
gps_sbittip_tb12= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-12_sbittip_tom_gps.csv')
gps_sbittip_tb12.epoch= pd.to_datetime(gps_sbittip_tb12.epoch, unit= 's', utc=True)
gps_sbittip_tb12.set_index('epoch', inplace=True)

# helihoist
gps_helih_tb12= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-12_helihoist-1_tom_gps.csv')
gps_helih_tb12.epoch= pd.to_datetime(gps_helih_tb12.epoch, unit= 's', utc=True)
gps_helih_tb12.set_index('epoch', inplace=True)

In [13]:
# READ CSV files of GPS Data: turbine 13

# sbitroot
gps_sbitroot_tb13= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-13_sbitroot_tom_gps.csv')
gps_sbitroot_tb13.epoch= pd.to_datetime(gps_sbitroot_tb13.epoch, unit= 's', utc=True)
gps_sbitroot_tb13.set_index('epoch', inplace=True)

# sbittip
gps_sbittip_tb13= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-13_sbittip_tom_gps.csv')
gps_sbittip_tb13.epoch= pd.to_datetime(gps_sbittip_tb13.epoch, unit= 's', utc=True)
gps_sbittip_tb13.set_index('epoch', inplace=True)

# helihoist
gps_helih_tb13= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-13_helihoist-1_tom_gps.csv')
gps_helih_tb13.epoch= pd.to_datetime(gps_helih_tb13.epoch, unit= 's', utc=True)
gps_helih_tb13.set_index('epoch', inplace=True)

In [14]:
# READ CSV files of GPS Data: turbine 14

# sbitroot
gps_sbitroot_tb14= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-14_sbitroot_tom_gps.csv')
gps_sbitroot_tb14.epoch= pd.to_datetime(gps_sbitroot_tb14.epoch, unit= 's', utc=True)
gps_sbitroot_tb14.set_index('epoch', inplace=True)

# sbittip
gps_sbittip_tb14= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-14_sbittip_tom_gps.csv')
gps_sbittip_tb14.epoch= pd.to_datetime(gps_sbittip_tb14.epoch, unit= 's', utc=True)
gps_sbittip_tb14.set_index('epoch', inplace=True)

# helihoist
gps_helih_tb14= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-14_helihoist-1_tom_gps.csv')
gps_helih_tb14.epoch= pd.to_datetime(gps_helih_tb14.epoch, unit= 's', utc=True)
gps_helih_tb14.set_index('epoch', inplace=True)

In [15]:
# READ CSV files of GPS Data: turbine 16

# sbitroot
gps_sbitroot_tb16= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-16_sbitroot_tom_gps.csv')
gps_sbitroot_tb16.epoch= pd.to_datetime(gps_sbitroot_tb16.epoch, unit= 's', utc=True)
gps_sbitroot_tb16.set_index('epoch', inplace=True)

# sbittip
gps_sbittip_tb16= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-16_sbittip_tom_gps.csv')
gps_sbittip_tb16.epoch= pd.to_datetime(gps_sbittip_tb16.epoch, unit= 's', utc=True)
gps_sbittip_tb16.set_index('epoch', inplace=True)

# helihoist
gps_helih_tb16= pd.read_csv('/Volumes/MASTERTHESE/MasterThese/GPS/turbine-16_helihoist-1_tom_gps.csv')
gps_helih_tb16.epoch= pd.to_datetime(gps_helih_tb16.epoch, unit= 's', utc=True)
gps_helih_tb16.set_index('epoch', inplace=True)

## Oscillation data

In [16]:
def concat_data(path):
    # concatenate files
    # indicate the correct path
    # files to concatenate must be in the same directory
    
    files = glob(path + "/*.csv")

    data = []

    for filename in files:
        df = pd.read_csv(filename, index_col=None, header=0)
        data.append(df)

    frame = pd.concat(data, axis=0, ignore_index=True)

    frame.epoch= pd.to_datetime(frame.epoch, unit= 's', utc=True)
    frame.set_index('epoch', inplace=True)
    frame = frame.sort_index()
    

    return frame

In [17]:
# TB 4

path_tb4_hh = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb4/Blade_Landings/Helihoist'
hh_frame_tb4 = concat_data(path_tb4_hh)

path_tb4_sbitroot = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb4/Blade_Landings/sbitroot'
sbitroot_frame_tb4 = concat_data(path_tb4_sbitroot)

In [18]:
# TB 5

path_tb5_hh = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb5/Blade_Landings/Helihoist'
hh_frame_tb5 = concat_data(path_tb5_hh)

path_tb5_sbitroot = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb5/Blade_Landings/sbitroot'
sbitroot_frame_tb5 = concat_data(path_tb5_sbitroot)

In [19]:
# TB 6

path_tb6_hh = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb6/Blade_Landings/Helihoist'
hh_frame_tb6 = concat_data(path_tb6_hh)

path_tb6_sbitroot = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb6/Blade_Landings/sbitroot'
sbitroot_frame_tb6 = concat_data(path_tb6_sbitroot)

In [20]:
# TB 7

path_tb7_hh = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb7/Blade_Landings/Helihoist'
hh_frame_tb7 = concat_data(path_tb7_hh)

path_tb7_sbitroot = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb7/Blade_Landings/sbitroot'
sbitroot_frame_tb7 = concat_data(path_tb7_sbitroot)

In [21]:
# TB 8 

path_tb8_hh = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb8/Blade_Landings/Helihoist'
hh_frame_tb8 = concat_data(path_tb8_hh)

path_tb8_sbitroot = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb8/Blade_Landings/sbitroot'
sbitroot_frame_tb8 = concat_data(path_tb8_sbitroot)

In [22]:
# TB 10

path_tb10_hh = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb10/Blade_Landings/Helihoist'
hh_frame_tb10 = concat_data(path_tb10_hh)

path_tb10_sbitroot = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb10/Blade_Landings/sbitroot'
sbitroot_frame_tb10 = concat_data(path_tb10_sbitroot)

In [23]:
# TB 11

path_tb11_hh = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb11/Blade_Landings/Helihoist'
hh_frame_tb11 = concat_data(path_tb11_hh)

path_tb11_sbitroot = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb11/Blade_Landings/sbitroot'
sbitroot_frame_tb11 = concat_data(path_tb11_sbitroot)

In [24]:
# TB 12

path_tb12_hh = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb12/Blade_Landings/Helihoist'
hh_frame_tb12 = concat_data(path_tb12_hh)

path_tb12_sbitroot = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb12/Blade_Landings/sbitroot'
sbitroot_frame_tb12 = concat_data(path_tb12_sbitroot)

In [25]:
# TB 13

path_tb13_hh = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb13/Blade_Landings/Helihoist'
hh_frame_tb13 = concat_data(path_tb13_hh)

path_tb13_sbitroot = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb13/Blade_Landings/sbitroot'
sbitroot_frame_tb13 = concat_data(path_tb13_sbitroot)

In [26]:
# TB 14

path_tb14_hh = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb14/Blade_Landings/Helihoist'
hh_frame_tb14 = concat_data(path_tb14_hh)

path_tb14_sbitroot = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb14/Blade_Landings/sbitroot'
sbitroot_frame_tb14 = concat_data(path_tb14_sbitroot)

In [27]:
# TB 16

path_tb16_hh = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb16/Blade_Landings/Helihoist'
hh_frame_tb16 = concat_data(path_tb16_hh)

path_tb16_sbitroot = r'/Volumes/MASTERTHESE/MasterThese/turbines/tb16/Blade_Landings/sbitroot'
sbitroot_frame_tb16 = concat_data(path_tb16_sbitroot)

# Functions

In [28]:
def find_installationperiod(data_hh, data_sbitroot, data_sbittip):
    # Yield indices of installtion period.
    # Use altitude of dataset
    
    hub_height = 90 # hub height = 90m

    # smoothing the time series: cut the outliers and find the 20min mean value
    hh, sr, st = (data_hh.resample('1s').mean().rolling('20min').mean(), data_sbitroot.resample('1s').mean().rolling('20min').mean(), data_sbittip.resample('1s').mean().rolling('20min').mean())

    hh_sbi = hh[(hh > hub_height) & (sr > hub_height) & (st > hub_height)]
    sr_sbi = sr[(hh > hub_height) & (sr > hub_height) & (st > hub_height)]
    st_sbi = st[(hh > hub_height) & (sr > hub_height) & (st > hub_height)]

    # determine start and end period of single blade installations
    inst_period = sr_sbi[sr_sbi.index.to_series().diff() > pd.to_timedelta(60, unit='s')].index.to_list()
    inst_period.insert(0, sr_sbi.index[0])
    inst_period.insert(len(inst_period), sr_sbi.index[-1])
    sbi_indices = list()

    for i in range(len(inst_period) - 1):
        sbi_indices.append((inst_period[i], hh_sbi[inst_period[i]:inst_period[i+1]].index[-2]))
    
    return sbi_indices

In [29]:
def divide_chunks(lst, n):
    #Yield successive n-sized chunks from lst.
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [30]:
def find_impacts(acc_data, acc_data_check, acc_threshold, acc_threshold_check, time_limit, min_impact_time):
    # acc_data = acceleration data
    # acc_data_check = acceleration data to validate if it is really an impact
    # acc_threshold = acceleration threshold above which an acceleration is declared as an impact
    # acc_threshold_check = acceleration threshold above which an acceleration is declared as an impact (validation data)
    # time_limit = min. time between two impacts to avoid correlation
    # min_impact_time = min. impact duration
    
    
    # find start values of impact event

    sbitroot_ind = acc_data[(abs(acc_data) > acc_threshold) & (abs(acc_data_check) > acc_threshold_check)]

    # find the indices where the difference is bigger than the time limit (avoid correlation)

    deltaT_startImpact = sbitroot_ind[sbitroot_ind.index.to_series().diff() > time_limit].index.to_list()

    # find the points in time where the threshold is exceeded (start of the impact event)
    # add the start of the first impact event
    deltaT_startImpact.insert(0, sbitroot_ind.index[0])
    
    # find end values of impact event

    sbitroot_ind_inverse = sbitroot_ind.sort_index(ascending=False)

    # find the indices where the difference is bigger than the time limit (avoid correlation)

    deltaT_endImpact = sbitroot_ind_inverse[sbitroot_ind_inverse.index.to_series().diff() < -time_limit].index.to_list()

    # find the points in time where the threshold is exceeded (start of the impact event)
    # add the end of the last impact event
    deltaT_endImpact.insert(0, sbitroot_ind_inverse.index[0])
    deltaT_endImpact.sort(reverse=False)
    
    
    
    impact_times = list()
    for i in range(len(deltaT_endImpact)):
        if (deltaT_endImpact[i]- deltaT_startImpact[i]) > min_impact_time:
            # filter out measurement errors
            impact_times.append((deltaT_startImpact[i], deltaT_endImpact[i]))

    return impact_times

In [31]:
def find_indep_oscillations(sbi_indices, impact_times, wind_data_index, data):
    # sbi_indices = single blade installation times indices 
    # impact_times = impact indices
    # wind_data_index = index of wind data
    # data = the dataset where to find independent time periods
    
    # Find start values of sbi and impacts

    sbi_indices_start = [i[0] for i in sbi_indices]
    impact_times_start = [i[0] for i in impact_times]

    # Remove all impact starts after the end of the environmental data set, so that the deflection data set is of 
    #equal length to wind dataset. 

    impact_times_start= [x for x in impact_times_start if x < wind_data_index[-1]]


    # if the last first impact of the installation attempt gets cut off due to the shorter environmental data set, 
    # then add the last Datetime of environmental data in impacts (treated as impact event)

    if impact_times_start[-1]< sbi_indices_start[-1]:
        impact_times_start.append(wind_data_index[-1])
    
    # identify period from start sbi-period to first impact = INDEPENDENT OSCILLATION
    # write first impacts after start of blade installation into list
    # to identify independent oscillations, timestamp of impact needs to be bigger than timestamp of sbi-period
    # timestamp of impact needs to be the first impact after sbi-period

    # indices of first impacts
    first_impacts = []
    # timestamp of first impacts
    first_impacts_start = []

    for i in range (len(sbi_indices_start)):
        first_impacts.append(next(x for x, val in enumerate(impact_times_start)
                                      if val > sbi_indices_start[i] ))


    for i in range(len(sbi_indices_start)):
        first_impacts_start.append(impact_times_start[first_impacts[i]])
    
    independent_osc = []
    for i in range (len(sbi_indices_start)):
            independent_osc.append(data[sbi_indices_start[i]:impact_times_start[first_impacts[i]]])
    
    
    
    return independent_osc, sbi_indices_start, first_impacts_start

In [32]:
def block_maxima(data, sbi_indices_start, first_impacts_start):
    
    # calculate number of chunks for each independent interval 
    # number of chunks is the length of the independent interval divided by the time interval
    # therfore, the start value minus the end value of the independent interval is the time difference 

    time_interval = pd.to_timedelta(60, unit='s')

    indep_duration = []
    for i in range(len(sbi_indices_start)):
        indep_duration.append(first_impacts_start[i] - sbi_indices_start[i])

    num_chunks = []
    for i in range(len(sbi_indices_start)):
        num_chunks.append(indep_duration[i]/time_interval)
    
    
    # Determine the chunk size in each independent time interval 

    chunk_size_deflection = []
    for i in range(len(sbi_indices_start)):
        chunk_size_deflection.append(len(data[i])/ num_chunks[i])
        chunk_size_deflection[i] = math.floor(chunk_size_deflection[i])

    chunk_size_deflection = min(chunk_size_deflection) 
    
    # Create list with chunks of independent intervals 
    # Independent oscillations are split up into chunks
    # indep_blocks[i][j], here i indicates the installation attempt j indicates the blocks in each installation attempt

    indep_blocks = []
    for i in range(len(sbi_indices_start)):
        indep_blocks.append(list(divide_chunks(data[i], chunk_size_deflection)))
        
    # Determine block maxima, blocks that does not contain full 60 sec of data are filtered out. 

    block_maxima = []

    for i in range(len(sbi_indices_start)):
        for j in range(len(indep_blocks[i])):
            if len(indep_blocks[i][j]) == len(indep_blocks[0][0]): 
                block_maxima.append((indep_blocks[i][j].idxmax(), max(indep_blocks[i][j]) ))
    
    
    maxima_values = []
    for i in range (len(block_maxima)):
        maxima_values.append(block_maxima[i][1])
    
    return maxima_values

In [33]:
def block_mean(data, sbi_indices_start, first_impacts_start):
    # Determine the chunk size in each independent time interval 

    
    time_interval = pd.to_timedelta(60, unit='s')

    indep_duration = []
    for i in range(len(sbi_indices_start)):
        indep_duration.append(first_impacts_start[i] - sbi_indices_start[i])

    num_chunks = []
    for i in range(len(sbi_indices_start)):
        num_chunks.append(indep_duration[i]/time_interval)
    
    chunk_size_wind = []
    for i in range(len(sbi_indices_start)):
        chunk_size_wind.append(len(data[i])/ num_chunks[i])
        chunk_size_wind[i] = math.floor(chunk_size_wind[i])
        #chunk_size_wind[i] = math.ceil(chunk_size_wind[i])

    chunk_size_wind = min(chunk_size_wind) 
    
    # Create list with chunks of independent intervals 
    # Independent oscillations are split up into chunks
    # indep_blocks[i][j], here i indicates the installation attempt j indicates the blocks in each installation attempt

    indep_blocks_wind = []
    for i in range(len(sbi_indices_start)):
        indep_blocks_wind.append(list(divide_chunks(data[i], chunk_size_wind)))
        
        
    # Determine block mean, blocks that does not contain full 60 sec of data are filtered out. 

    block_mean = []

    for i in range(len(sbi_indices_start)):
        for j in range(len(indep_blocks_wind[i])):
            if len(indep_blocks_wind[i][j]) == len(indep_blocks_wind[0][0]): 
                block_mean.append((indep_blocks_wind[i][j].mean() ))

    return block_mean

# Installtion periods

In [34]:
sbi_indices_tb4= find_installationperiod(gps_helih_tb4.altitude, gps_sbitroot_tb4.altitude, gps_sbittip_tb4.altitude)
sbi_indices_tb5= find_installationperiod(gps_helih_tb5.altitude, gps_sbitroot_tb5.altitude, gps_sbittip_tb5.altitude)
sbi_indices_tb6= find_installationperiod(gps_helih_tb6.altitude, gps_sbitroot_tb6.altitude, gps_sbittip_tb6.altitude)
#sbi_indices_tb7= find_installationperiod(gps_helih_tb7.altitude, gps_sbitroot_tb7.altitude, gps_sbittip_tb7.altitude)
sbi_indices_tb8= find_installationperiod(gps_helih_tb8.altitude, gps_sbitroot_tb8.altitude, gps_sbittip_tb8.altitude)
sbi_indices_tb10= find_installationperiod(gps_helih_tb10.altitude, gps_sbitroot_tb10.altitude, gps_sbittip_tb10.altitude)
sbi_indices_tb11= find_installationperiod(gps_helih_tb11.altitude, gps_sbitroot_tb11.altitude, gps_sbittip_tb11.altitude)
sbi_indices_tb12= find_installationperiod(gps_helih_tb12.altitude, gps_sbitroot_tb12.altitude, gps_sbittip_tb12.altitude)
#sbi_indices_tb13= find_installationperiod(gps_helih_tb13.altitude, gps_sbitroot_tb13.altitude, gps_sbittip_tb13.altitude)
sbi_indices_tb14= find_installationperiod(gps_helih_tb14.altitude, gps_sbitroot_tb14.altitude, gps_sbittip_tb14.altitude)
#sbi_indices_tb16= find_installationperiod(gps_helih_tb16.altitude, gps_sbitroot_tb16.altitude, gps_sbittip_tb16.altitude)

# IMPACTS

## Filter outliers from accelerations X and Z

In [35]:
# TB 4

# SBIT root 
median_x_tb4 = sbitroot_frame_tb4.loc[sbitroot_frame_tb4['acc_x']< abs(10), 'acc_x'].median()
median_z_tb4 = sbitroot_frame_tb4.loc[sbitroot_frame_tb4['acc_z']< abs(10), 'acc_z'].median()

sbitroot_frame_tb4['acc_x'] = np.where(sbitroot_frame_tb4["acc_x"] > abs(10), median_x_tb4,sbitroot_frame_tb4['acc_x'])
sbitroot_frame_tb4['acc_z'] = np.where(sbitroot_frame_tb4["acc_z"] > abs(10), median_z_tb4,sbitroot_frame_tb4['acc_z'])

hh_frame_tb4['acc_x'] = np.where(hh_frame_tb4["acc_x"] > abs(10), median_x_tb4, hh_frame_tb4['acc_x'])
hh_frame_tb4['acc_z'] = np.where(hh_frame_tb4["acc_z"] > abs(10), median_z_tb4, hh_frame_tb4['acc_z'])

In [36]:
# TB 5

# SBIT root 
median_x_tb5 = sbitroot_frame_tb5.loc[sbitroot_frame_tb5['acc_x']< abs(10), 'acc_x'].median()
median_z_tb5 = sbitroot_frame_tb5.loc[sbitroot_frame_tb5['acc_z']< abs(10), 'acc_z'].median()

sbitroot_frame_tb5['acc_x'] = np.where(sbitroot_frame_tb5["acc_x"] > abs(10), median_x_tb5,sbitroot_frame_tb5['acc_x'])
sbitroot_frame_tb5['acc_z'] = np.where(sbitroot_frame_tb5["acc_z"] > abs(10), median_z_tb5,sbitroot_frame_tb5['acc_z'])

hh_frame_tb5['acc_x'] = np.where(hh_frame_tb5["acc_x"] > abs(10), median_x_tb5, hh_frame_tb5['acc_x'])
hh_frame_tb5['acc_z'] = np.where(hh_frame_tb5["acc_z"] > abs(10), median_z_tb5, hh_frame_tb5['acc_z'])

In [37]:
# TB 6

# SBIT root 
median_x_tb6 = sbitroot_frame_tb6.loc[sbitroot_frame_tb6['acc_x']< abs(10), 'acc_x'].median()
median_z_tb6 = sbitroot_frame_tb6.loc[sbitroot_frame_tb6['acc_z']< abs(10), 'acc_z'].median()

sbitroot_frame_tb6['acc_x'] = np.where(sbitroot_frame_tb6["acc_x"] > abs(10), median_x_tb6,sbitroot_frame_tb6['acc_x'])
sbitroot_frame_tb6['acc_z'] = np.where(sbitroot_frame_tb6["acc_z"] > abs(10), median_z_tb6,sbitroot_frame_tb6['acc_z'])

hh_frame_tb6['acc_x'] = np.where(hh_frame_tb6["acc_x"] > abs(10), median_x_tb6, hh_frame_tb6['acc_x'])
hh_frame_tb6['acc_z'] = np.where(hh_frame_tb6["acc_z"] > abs(10), median_z_tb6, hh_frame_tb6['acc_z'])

In [38]:
# TB 7

# SBIT root 
median_x_tb7 = sbitroot_frame_tb7.loc[sbitroot_frame_tb7['acc_x']< abs(10), 'acc_x'].median()
median_z_tb7 = sbitroot_frame_tb7.loc[sbitroot_frame_tb7['acc_z']< abs(10), 'acc_z'].median()

sbitroot_frame_tb7['acc_x'] = np.where(sbitroot_frame_tb7["acc_x"] > abs(10), median_x_tb7,sbitroot_frame_tb7['acc_x'])
sbitroot_frame_tb7['acc_z'] = np.where(sbitroot_frame_tb7["acc_z"] > abs(10), median_z_tb7,sbitroot_frame_tb7['acc_z'])

hh_frame_tb7['acc_x'] = np.where(hh_frame_tb7["acc_x"] > abs(10), median_x_tb7, hh_frame_tb7['acc_x'])
hh_frame_tb7['acc_z'] = np.where(hh_frame_tb7["acc_z"] > abs(10), median_z_tb7, hh_frame_tb7['acc_z'])

In [39]:
# TB 8

# SBIT root 
median_x = sbitroot_frame_tb8.loc[sbitroot_frame_tb8['acc_x']< abs(10), 'acc_x'].median()
median_z = sbitroot_frame_tb8.loc[sbitroot_frame_tb8['acc_z']< abs(10), 'acc_z'].median()

sbitroot_frame_tb8['acc_x'] = np.where(sbitroot_frame_tb8["acc_x"] > abs(10), median_x,sbitroot_frame_tb8['acc_x'])
sbitroot_frame_tb8['acc_z'] = np.where(sbitroot_frame_tb8["acc_z"] > abs(10), median_z,sbitroot_frame_tb8['acc_z'])

hh_frame_tb8['acc_x'] = np.where(hh_frame_tb8["acc_x"] > abs(10), median_x, hh_frame_tb8['acc_x'])
hh_frame_tb8['acc_z'] = np.where(hh_frame_tb8["acc_z"] > abs(10), median_z, hh_frame_tb8['acc_z'])

In [40]:
# TB 10

# SBIT root 
median_x_tb10 = sbitroot_frame_tb10.loc[sbitroot_frame_tb10['acc_x']< abs(10), 'acc_x'].median()
median_z_tb10 = sbitroot_frame_tb10.loc[sbitroot_frame_tb10['acc_z']< abs(10), 'acc_z'].median()

sbitroot_frame_tb10['acc_x'] = np.where(sbitroot_frame_tb10["acc_x"] > abs(10), median_x_tb10,sbitroot_frame_tb10['acc_x'])
sbitroot_frame_tb10['acc_z'] = np.where(sbitroot_frame_tb10["acc_z"] > abs(10), median_z_tb10,sbitroot_frame_tb10['acc_z'])

hh_frame_tb10['acc_x'] = np.where(hh_frame_tb10["acc_x"] > abs(10), median_x_tb10, hh_frame_tb10['acc_x'])
hh_frame_tb10['acc_z'] = np.where(hh_frame_tb10["acc_z"] > abs(10), median_z_tb10, hh_frame_tb10['acc_z'])

In [41]:
# TB 11

# SBIT root 
median_x_tb11 = sbitroot_frame_tb11.loc[sbitroot_frame_tb11['acc_x']< abs(10), 'acc_x'].median()
median_z_tb11 = sbitroot_frame_tb11.loc[sbitroot_frame_tb11['acc_z']< abs(10), 'acc_z'].median()

sbitroot_frame_tb11['acc_x'] = np.where(sbitroot_frame_tb11["acc_x"] > abs(10), median_x_tb11,sbitroot_frame_tb11['acc_x'])
sbitroot_frame_tb11['acc_z'] = np.where(sbitroot_frame_tb11["acc_z"] > abs(10), median_z_tb11,sbitroot_frame_tb11['acc_z'])

hh_frame_tb11['acc_x'] = np.where(hh_frame_tb11["acc_x"] > abs(10), median_x_tb11, hh_frame_tb11['acc_x'])
hh_frame_tb11['acc_z'] = np.where(hh_frame_tb11["acc_z"] > abs(10), median_z_tb11, hh_frame_tb11['acc_z'])

In [42]:
# TB 12

# SBIT root 
median_x_tb12 = sbitroot_frame_tb12.loc[sbitroot_frame_tb12['acc_x']< abs(10), 'acc_x'].median()
median_z_tb12 = sbitroot_frame_tb12.loc[sbitroot_frame_tb12['acc_z']< abs(10), 'acc_z'].median()

sbitroot_frame_tb12['acc_x'] = np.where(sbitroot_frame_tb12["acc_x"] > abs(10), median_x_tb12,sbitroot_frame_tb12['acc_x'])
sbitroot_frame_tb12['acc_z'] = np.where(sbitroot_frame_tb12["acc_z"] > abs(10), median_z_tb12,sbitroot_frame_tb12['acc_z'])

hh_frame_tb12['acc_x'] = np.where(hh_frame_tb12["acc_x"] > abs(10), median_x_tb12, hh_frame_tb12['acc_x'])
hh_frame_tb12['acc_z'] = np.where(hh_frame_tb12["acc_z"] > abs(10), median_z_tb12, hh_frame_tb12['acc_z'])

In [43]:
# TB 13

# SBIT root 
median_x_tb13 = sbitroot_frame_tb13.loc[sbitroot_frame_tb13['acc_x']< abs(10), 'acc_x'].median()
median_z_tb13 = sbitroot_frame_tb13.loc[sbitroot_frame_tb13['acc_z']< abs(10), 'acc_z'].median()

sbitroot_frame_tb13['acc_x'] = np.where(sbitroot_frame_tb13["acc_x"] > abs(10), median_x_tb13,sbitroot_frame_tb13['acc_x'])
sbitroot_frame_tb13['acc_z'] = np.where(sbitroot_frame_tb13["acc_z"] > abs(10), median_z_tb13,sbitroot_frame_tb13['acc_z'])

hh_frame_tb13['acc_x'] = np.where(hh_frame_tb13["acc_x"] > abs(10), median_x_tb13, hh_frame_tb13['acc_x'])
hh_frame_tb13['acc_z'] = np.where(hh_frame_tb13["acc_z"] > abs(10), median_z_tb13, hh_frame_tb13['acc_z'])

In [44]:
# TB 14

# SBIT root 
median_x_tb14 = sbitroot_frame_tb14.loc[sbitroot_frame_tb14['acc_x']< abs(10), 'acc_x'].median()
median_z_tb14 = sbitroot_frame_tb14.loc[sbitroot_frame_tb14['acc_z']< abs(10), 'acc_z'].median()

sbitroot_frame_tb14['acc_x'] = np.where(sbitroot_frame_tb14["acc_x"] > abs(10), median_x_tb14,sbitroot_frame_tb14['acc_x'])
sbitroot_frame_tb14['acc_z'] = np.where(sbitroot_frame_tb14["acc_z"] > abs(10), median_z_tb14,sbitroot_frame_tb14['acc_z'])

hh_frame_tb14['acc_x'] = np.where(hh_frame_tb14["acc_x"] > abs(10), median_x_tb14, hh_frame_tb14['acc_x'])
hh_frame_tb14['acc_z'] = np.where(hh_frame_tb14["acc_z"] > abs(10), median_z_tb14, hh_frame_tb14['acc_z'])

In [45]:
# TB 16

# SBIT root 
median_x_tb16 = sbitroot_frame_tb16.loc[sbitroot_frame_tb16['acc_x']< abs(10), 'acc_x'].median()
median_z_tb16 = sbitroot_frame_tb16.loc[sbitroot_frame_tb16['acc_z']< abs(10), 'acc_z'].median()

sbitroot_frame_tb16['acc_x'] = np.where(sbitroot_frame_tb16["acc_x"] > abs(10), median_x_tb16,sbitroot_frame_tb16['acc_x'])
sbitroot_frame_tb16['acc_z'] = np.where(sbitroot_frame_tb16["acc_z"] > abs(10), median_z_tb16,sbitroot_frame_tb16['acc_z'])

hh_frame_tb16['acc_x'] = np.where(hh_frame_tb16["acc_x"] > abs(10), median_x_tb16, hh_frame_tb16['acc_x'])
hh_frame_tb16['acc_z'] = np.where(hh_frame_tb16["acc_z"] > abs(10), median_z_tb16, hh_frame_tb16['acc_z'])

## Determine installation period (blade at HH)

In [46]:
# TB 4

sbitroot_installation_tb4 = pd.DataFrame()
hh_installation_tb4= pd.DataFrame()

for start, end in sbi_indices_tb4:
    sbitroot_installation_tb4 = sbitroot_installation_tb4.append(sbitroot_frame_tb4[start:end])
    hh_installation_tb4 = hh_installation_tb4.append(hh_frame_tb4[start:end])

In [47]:
# TB 5

sbitroot_installation_tb5 = pd.DataFrame()
hh_installation_tb5= pd.DataFrame()

for start, end in sbi_indices_tb5:
    sbitroot_installation_tb5 = sbitroot_installation_tb5.append(sbitroot_frame_tb5[start:end])
    hh_installation_tb5 = hh_installation_tb5.append(hh_frame_tb5[start:end])

In [48]:
# TB 6

sbitroot_installation_tb6 = pd.DataFrame()
hh_installation_tb6= pd.DataFrame()

for start, end in sbi_indices_tb6:
    sbitroot_installation_tb6 = sbitroot_installation_tb6.append(sbitroot_frame_tb6[start:end])
    hh_installation_tb6 = hh_installation_tb6.append(hh_frame_tb6[start:end])

In [49]:
# TB 7

#sbitroot_installation_tb7 = pd.DataFrame()

#for start, end in sbi_indices_tb7:
    #sbitroot_installation_tb7 = sbitroot_installation_tb7.append(sbitroot_frame_tb7[start:end])

In [50]:
# TB 8

sbitroot_installation_tb8 = pd.DataFrame()
hh_installation_tb8= pd.DataFrame()

for start, end in sbi_indices_tb8:
    sbitroot_installation_tb8 = sbitroot_installation_tb8.append(sbitroot_frame_tb8[start:end])
    hh_installation_tb8 = hh_installation_tb8.append(hh_frame_tb8[start:end])

In [51]:
# TB 10

sbitroot_installation_tb10 = pd.DataFrame()
hh_installation_tb10= pd.DataFrame()

for start, end in sbi_indices_tb10:
    sbitroot_installation_tb10 = sbitroot_installation_tb10.append(sbitroot_frame_tb10[start:end])
    hh_installation_tb10 = hh_installation_tb10.append(hh_frame_tb10[start:end])

In [52]:
# TB 11

sbitroot_installation_tb11 = pd.DataFrame()
hh_installation_tb11= pd.DataFrame()

for start, end in sbi_indices_tb11:
    sbitroot_installation_tb11 = sbitroot_installation_tb11.append(sbitroot_frame_tb11[start:end])
    hh_installation_tb11 = hh_installation_tb11.append(hh_frame_tb11[start:end])

In [53]:
# TB 12

sbitroot_installation_tb12 = pd.DataFrame()
hh_installation_tb12= pd.DataFrame()

for start, end in sbi_indices_tb12:
    sbitroot_installation_tb12 = sbitroot_installation_tb12.append(sbitroot_frame_tb12[start:end])
    hh_installation_tb12 = hh_installation_tb12.append(hh_frame_tb12[start:end])

In [54]:
# TB 13

#sbitroot_installation_tb13 = pd.DataFrame()


#for start, end in sbi_indices_tb13:
    #sbitroot_installation_tb13 = sbitroot_installation_tb13.append(sbitroot_frame_tb13[start:end])

In [55]:
# TB 14

sbitroot_installation_tb14 = pd.DataFrame()
hh_installation_tb14= pd.DataFrame()

for start, end in sbi_indices_tb14:
    sbitroot_installation_tb14 = sbitroot_installation_tb14.append(sbitroot_frame_tb14[start:end])
    hh_installation_tb14 = hh_installation_tb14.append(hh_frame_tb14[start:end])

In [56]:
# TB 16

#sbitroot_installation_tb16 = pd.DataFrame()

#for start, end in sbi_indices_tb16:
    #sbitroot_installation_tb16 = sbitroot_installation_tb16.append(sbitroot_frame_tb16[start:end])

## Impact thresholds

In [57]:
# Find independent oscillation periods

acc_threshold_x = 0.5 #acceleration threshold (m/s^2)
acc_threshold_z = 0.5
acc_threshold_y = 0.3

limit = pd.to_timedelta(60, unit='s') #time limit to avoid correlation
min_impact_time = pd.to_timedelta(10, unit='s') # minimum time of impact event, erasing measurment errors

## List of analysed turbines

In [58]:
list_of_turbines= [4,5,6,8,10,11,12,14]
print(list_of_turbines)

[4, 5, 6, 8, 10, 11, 12, 14]


## Impacts in X Direction

In [59]:
impact_times_tb4_x= find_impacts(sbitroot_installation_tb4.acc_x, sbitroot_installation_tb4.acc_y, acc_threshold_x, acc_threshold_y, limit, min_impact_time)

In [60]:
impact_times_tb5_x= find_impacts(sbitroot_installation_tb5.acc_x, sbitroot_installation_tb5.acc_y, acc_threshold_x, acc_threshold_y, limit, min_impact_time)

In [61]:
impact_times_tb6_x= find_impacts(sbitroot_installation_tb6.acc_x, sbitroot_installation_tb6.acc_y, acc_threshold_x, acc_threshold_y, limit, min_impact_time)

In [62]:
impact_times_tb8_x= find_impacts(sbitroot_installation_tb8.acc_x, sbitroot_installation_tb8.acc_y, acc_threshold_x, acc_threshold_y, limit, min_impact_time)

In [63]:
impact_times_tb10_x= find_impacts(sbitroot_installation_tb10.acc_x, sbitroot_installation_tb10.acc_y, acc_threshold_x, acc_threshold_y, limit, min_impact_time)

In [64]:
impact_times_tb11_x= find_impacts(sbitroot_installation_tb11.acc_x, sbitroot_installation_tb11.acc_y, acc_threshold_x, acc_threshold_y, limit, min_impact_time)

In [65]:
impact_times_tb12_x= find_impacts(sbitroot_installation_tb12.acc_x, sbitroot_installation_tb12.acc_y, acc_threshold_x, acc_threshold_y, limit, min_impact_time)

In [66]:
impact_times_tb14_x= find_impacts(sbitroot_installation_tb14.acc_x, sbitroot_installation_tb14.acc_y, acc_threshold_x, acc_threshold_y, limit, min_impact_time)

## Impacts in Z Direction

In [67]:
impact_times_tb4_z= find_impacts(sbitroot_installation_tb4.acc_z, sbitroot_installation_tb4.acc_y, acc_threshold_z, acc_threshold_y, limit, min_impact_time)

In [68]:
impact_times_tb5_z= find_impacts(sbitroot_installation_tb5.acc_z, sbitroot_installation_tb5.acc_y, acc_threshold_z, acc_threshold_y, limit, min_impact_time)

In [69]:
impact_times_tb6_z= find_impacts(sbitroot_installation_tb6.acc_z, sbitroot_installation_tb6.acc_y, acc_threshold_z, acc_threshold_y, limit, min_impact_time)

In [70]:
impact_times_tb8_z= find_impacts(sbitroot_installation_tb8.acc_z, sbitroot_installation_tb8.acc_y, acc_threshold_z, acc_threshold_y, limit, min_impact_time)

In [71]:
impact_times_tb10_z= find_impacts(sbitroot_installation_tb10.acc_z, sbitroot_installation_tb10.acc_y, acc_threshold_z, acc_threshold_y, limit, min_impact_time)

In [72]:
impact_times_tb11_z= find_impacts(sbitroot_installation_tb11.acc_z, sbitroot_installation_tb11.acc_y, acc_threshold_z, acc_threshold_y, limit, min_impact_time)

In [73]:
impact_times_tb12_z= find_impacts(sbitroot_installation_tb12.acc_z, sbitroot_installation_tb12.acc_y, acc_threshold_z, acc_threshold_y, limit, min_impact_time)

In [74]:
impact_times_tb14_z= find_impacts(sbitroot_installation_tb14.acc_z, sbitroot_installation_tb14.acc_y, acc_threshold_z, acc_threshold_y, limit, min_impact_time)

## Impacts combined X and Z

In [75]:
impact_times_tb4 = impact_times_tb4_z + impact_times_tb4_x
impact_times_tb4.sort()

In [76]:
impact_times_tb5 = impact_times_tb5_z + impact_times_tb5_x
impact_times_tb5.sort()

In [77]:
impact_times_tb6 = impact_times_tb6_z + impact_times_tb6_x
impact_times_tb6.sort()

In [78]:
impact_times_tb8 = impact_times_tb8_z + impact_times_tb8_x
impact_times_tb8.sort()

In [79]:
impact_times_tb10 = impact_times_tb10_z + impact_times_tb10_x
impact_times_tb10.sort()

In [80]:
impact_times_tb11 = impact_times_tb11_z + impact_times_tb11_x
impact_times_tb11.sort()

In [81]:
impact_times_tb12 = impact_times_tb12_z + impact_times_tb12_x
impact_times_tb12.sort()

In [82]:
impact_times_tb14 = impact_times_tb14_z + impact_times_tb14_x
impact_times_tb14.sort()

# INDEPENDENT deflection SBIT

In [83]:
indep_deflection_tb4, sbi_start_tb4, impact_start_tb4= find_indep_oscillations(sbi_indices_tb4, impact_times_tb4, wind_data.index, sbitroot_installation_tb4.deflection )

In [84]:
indep_deflection_tb5, sbi_start_tb5, impact_start_tb5= find_indep_oscillations(sbi_indices_tb5, impact_times_tb5, wind_data.index, sbitroot_installation_tb5.deflection )

In [85]:
indep_deflection_tb6, sbi_start_tb6, impact_start_tb6= find_indep_oscillations(sbi_indices_tb6, impact_times_tb6, wind_data.index, sbitroot_installation_tb6.deflection )

In [86]:
indep_deflection_tb8, sbi_start_tb8, impact_start_tb8= find_indep_oscillations(sbi_indices_tb8, impact_times_tb8, wind_data.index, sbitroot_installation_tb8.deflection )

In [87]:
indep_deflection_tb10, sbi_start_tb10, impact_start_tb10= find_indep_oscillations(sbi_indices_tb10, impact_times_tb10, wind_data.index, sbitroot_installation_tb10.deflection )

In [88]:
indep_deflection_tb11, sbi_start_tb11, impact_start_tb11= find_indep_oscillations(sbi_indices_tb11, impact_times_tb11, wind_data.index, sbitroot_installation_tb11.deflection )

In [89]:
indep_deflection_tb12, sbi_start_tb12, impact_start_tb12= find_indep_oscillations(sbi_indices_tb12, impact_times_tb12, wind_data.index, sbitroot_installation_tb12.deflection )

In [90]:
indep_deflection_tb14, sbi_start_tb14, impact_start_tb14= find_indep_oscillations(sbi_indices_tb14, impact_times_tb14, wind_data.index, sbitroot_installation_tb14.deflection )

# INDEPENDENT deflection TOWER

In [91]:
indep_def_tower_tb4, sbi_start_tb4, impact_start_tb4= find_indep_oscillations(sbi_indices_tb4, impact_times_tb4, wind_data.index, hh_installation_tb4.deflection )

In [92]:
indep_def_tower_tb5, sbi_start_tb5, impact_start_tb5= find_indep_oscillations(sbi_indices_tb5, impact_times_tb5, wind_data.index, hh_installation_tb5.deflection )

In [93]:
indep_def_tower_tb6, sbi_start_tb6, impact_start_tb6= find_indep_oscillations(sbi_indices_tb6, impact_times_tb6, wind_data.index, hh_installation_tb6.deflection )

In [94]:
indep_def_tower_tb8, sbi_start_tb8, impact_start_tb8= find_indep_oscillations(sbi_indices_tb8, impact_times_tb8, wind_data.index, hh_installation_tb8.deflection )

In [95]:
indep_def_tower_tb10, sbi_start_tb10, impact_start_tb10= find_indep_oscillations(sbi_indices_tb10, impact_times_tb10, wind_data.index, hh_installation_tb10.deflection )

In [96]:
indep_def_tower_tb11, sbi_start_tb11, impact_start_tb11= find_indep_oscillations(sbi_indices_tb11, impact_times_tb11, wind_data.index, hh_installation_tb11.deflection )

In [97]:
indep_def_tower_tb12, sbi_start_tb12, impact_start_tb12= find_indep_oscillations(sbi_indices_tb12, impact_times_tb12, wind_data.index, hh_installation_tb12.deflection )

In [98]:
indep_def_tower_tb14, sbi_start_tb14, impact_start_tb14= find_indep_oscillations(sbi_indices_tb14, impact_times_tb14, wind_data.index, hh_installation_tb14.deflection )

# INDEPENDENT Environment

## Wind speed

In [99]:
indep_wind_tb4, sbi_start_tb4, impact_start_tb4 = find_indep_oscillations(sbi_indices_tb4, impact_times_tb4, wind_data.index, wind_data.wind_speed_0)

In [100]:
indep_wind_tb5, sbi_start_tb5, impact_start_tb5 = find_indep_oscillations(sbi_indices_tb5, impact_times_tb5, wind_data.index, wind_data.wind_speed_0)

In [101]:
indep_wind_tb6, sbi_start_tb6, impact_start_tb6 = find_indep_oscillations(sbi_indices_tb6, impact_times_tb6, wind_data.index, wind_data.wind_speed_0)

In [102]:
indep_wind_tb8, sbi_start_tb8, impact_start_tb8 = find_indep_oscillations(sbi_indices_tb8, impact_times_tb8, wind_data.index, wind_data.wind_speed_0)

In [103]:
indep_wind_tb10, sbi_start_tb10, impact_start_tb10 = find_indep_oscillations(sbi_indices_tb10, impact_times_tb10, wind_data.index, wind_data.wind_speed_0)

In [104]:
indep_wind_tb11, sbi_start_tb11, impact_start_tb11 = find_indep_oscillations(sbi_indices_tb11, impact_times_tb11, wind_data.index, wind_data.wind_speed_0)

In [105]:
indep_wind_tb12, sbi_start_tb12, impact_start_tb12 = find_indep_oscillations(sbi_indices_tb12, impact_times_tb12, wind_data.index, wind_data.wind_speed_0)

In [106]:
indep_wind_tb14, sbi_start_tb14, impact_start_tb14 = find_indep_oscillations(sbi_indices_tb14, impact_times_tb14, wind_data.index, wind_data.wind_speed_0)

## Wave height

In [107]:
indep_waveH_tb4, sbi_start_tb4, impact_start_tb4 = find_indep_oscillations(sbi_indices_tb4, impact_times_tb4, wave_data.index, wave_data.Hm0)

In [108]:
indep_waveH_tb5, sbi_start_tb5, impact_start_tb5 = find_indep_oscillations(sbi_indices_tb5, impact_times_tb5, wave_data.index, wave_data.Hm0)

In [109]:
indep_waveH_tb6, sbi_start_tb6, impact_start_tb6 = find_indep_oscillations(sbi_indices_tb6, impact_times_tb6, wave_data.index, wave_data.Hm0)

In [110]:
indep_waveH_tb8, sbi_start_tb8, impact_start_tb8 = find_indep_oscillations(sbi_indices_tb8, impact_times_tb8, wave_data.index, wave_data.Hm0)

In [111]:
indep_waveH_tb10, sbi_start_tb10, impact_start_tb10 = find_indep_oscillations(sbi_indices_tb10, impact_times_tb10, wave_data.index, wave_data.Hm0)

In [112]:
indep_waveH_tb11, sbi_start_tb11, impact_start_tb11 = find_indep_oscillations(sbi_indices_tb11, impact_times_tb11, wave_data.index, wave_data.Hm0)

In [113]:
indep_waveH_tb12, sbi_start_tb12, impact_start_tb12 = find_indep_oscillations(sbi_indices_tb12, impact_times_tb12, wave_data.index, wave_data.Hm0)

In [114]:
indep_waveH_tb14, sbi_start_tb14, impact_start_tb14 = find_indep_oscillations(sbi_indices_tb14, impact_times_tb14, wave_data.index, wave_data.Hm0)

## Peak Period

In [115]:
indep_waveTp_tb4, sbi_start_tb4, impact_start_tb4 = find_indep_oscillations(sbi_indices_tb4, impact_times_tb4, wave_data.index, wave_data.Tp)

In [116]:
indep_waveTp_tb5, sbi_start_tb5, impact_start_tb5 = find_indep_oscillations(sbi_indices_tb5, impact_times_tb5, wave_data.index, wave_data.Tp)

In [117]:
indep_waveTp_tb6, sbi_start_tb6, impact_start_tb6 = find_indep_oscillations(sbi_indices_tb6, impact_times_tb6, wave_data.index, wave_data.Tp)

In [118]:
indep_waveTp_tb8, sbi_start_tb8, impact_start_tb8 = find_indep_oscillations(sbi_indices_tb8, impact_times_tb8, wave_data.index, wave_data.Tp)

In [119]:
indep_waveTp_tb10, sbi_start_tb10, impact_start_tb10 = find_indep_oscillations(sbi_indices_tb10, impact_times_tb10, wave_data.index, wave_data.Tp)

In [120]:
indep_waveTp_tb11, sbi_start_tb11, impact_start_tb11 = find_indep_oscillations(sbi_indices_tb11, impact_times_tb11, wave_data.index, wave_data.Tp)

In [121]:
indep_waveTp_tb12, sbi_start_tb12, impact_start_tb12 = find_indep_oscillations(sbi_indices_tb12, impact_times_tb12, wave_data.index, wave_data.Tp)

In [122]:
indep_waveTp_tb14, sbi_start_tb14, impact_start_tb14 = find_indep_oscillations(sbi_indices_tb14, impact_times_tb14, wave_data.index, wave_data.Tp)

## Zero-up crossing period

In [123]:
indep_waveTz_tb4, sbi_start_tb4, impact_start_tb4 = find_indep_oscillations(sbi_indices_tb4, impact_times_tb4, wave_data.index, wave_data.Tz)

In [124]:
indep_waveTz_tb5, sbi_start_tb5, impact_start_tb5 = find_indep_oscillations(sbi_indices_tb5, impact_times_tb5, wave_data.index, wave_data.Tz)

In [125]:
indep_waveTz_tb6, sbi_start_tb6, impact_start_tb6 = find_indep_oscillations(sbi_indices_tb6, impact_times_tb6, wave_data.index, wave_data.Tz)

In [126]:
indep_waveTz_tb8, sbi_start_tb8, impact_start_tb8 = find_indep_oscillations(sbi_indices_tb8, impact_times_tb8, wave_data.index, wave_data.Tz)

In [127]:
indep_waveTz_tb10, sbi_start_tb10, impact_start_tb10 = find_indep_oscillations(sbi_indices_tb10, impact_times_tb10, wave_data.index, wave_data.Tz)

In [128]:
indep_waveTz_tb11, sbi_start_tb11, impact_start_tb11 = find_indep_oscillations(sbi_indices_tb11, impact_times_tb11, wave_data.index, wave_data.Tz)

In [129]:
indep_waveTz_tb12, sbi_start_tb12, impact_start_tb12 = find_indep_oscillations(sbi_indices_tb12, impact_times_tb12, wave_data.index, wave_data.Tz)

In [130]:
indep_waveTz_tb14, sbi_start_tb14, impact_start_tb14 = find_indep_oscillations(sbi_indices_tb14, impact_times_tb14, wave_data.index, wave_data.Tz)

# Block Maxima Deflection SBIT

In [131]:
maxima_values_tb4= block_maxima(indep_deflection_tb4, sbi_start_tb4, impact_start_tb4)

In [132]:
maxima_values_tb5= block_maxima(indep_deflection_tb5, sbi_start_tb5, impact_start_tb5)

In [133]:
maxima_values_tb6= block_maxima(indep_deflection_tb6, sbi_start_tb6, impact_start_tb6)

In [134]:
maxima_values_tb8= block_maxima(indep_deflection_tb8, sbi_start_tb8, impact_start_tb8)

In [135]:
maxima_values_tb10= block_maxima(indep_deflection_tb10, sbi_start_tb10, impact_start_tb10)

In [136]:
maxima_values_tb11= block_maxima(indep_deflection_tb11, sbi_start_tb11, impact_start_tb11)

In [137]:
maxima_values_tb12= block_maxima(indep_deflection_tb12, sbi_start_tb12, impact_start_tb12)

In [138]:
maxima_values_tb14= block_maxima(indep_deflection_tb14, sbi_start_tb14, impact_start_tb14)

# Block Maxima Deflection TOWER

In [139]:
maxima_val_tower_tb4= block_maxima(indep_def_tower_tb4, sbi_start_tb4, impact_start_tb4)

In [140]:
print(len(indep_def_tower_tb4[0]))
print(len(indep_deflection_tb4[0]))

106858
106862


In [141]:
maxima_val_tower_tb5= block_maxima(indep_def_tower_tb5, sbi_start_tb5, impact_start_tb5)

In [142]:
maxima_val_tower_tb6= block_maxima(indep_def_tower_tb6, sbi_start_tb6, impact_start_tb6)

In [143]:
maxima_val_tower_tb8= block_maxima(indep_def_tower_tb8, sbi_start_tb8, impact_start_tb8)

In [144]:
maxima_val_tower_tb10= block_maxima(indep_def_tower_tb10, sbi_start_tb10, impact_start_tb10)

In [145]:
maxima_val_tower_tb11= block_maxima(indep_def_tower_tb11, sbi_start_tb11, impact_start_tb11)

In [146]:
maxima_val_tower_tb12= block_maxima(indep_def_tower_tb12, sbi_start_tb12, impact_start_tb12)

In [147]:
maxima_val_tower_tb14= block_maxima(indep_def_tower_tb14, sbi_start_tb14, impact_start_tb14)

## Mean block wind speed
In case we use do not use 1- minute mean environmental data from the beginning, the following lines are needed. Here, the 1-minute mean environmental condition is calculated.

In [241]:
#mean_wind_tb4= block_mean(indep_wind_tb4, sbi_start_tb4, impact_start_tb4)

In [242]:
#mean_wind_tb5= block_mean(indep_wind_tb5, sbi_start_tb5, impact_start_tb5)

In [243]:
#mean_wind_tb6= block_mean(indep_wind_tb6, sbi_start_tb6, impact_start_tb6)

In [244]:
#mean_wind_tb8= block_mean(indep_wind_tb8, sbi_start_tb8, impact_start_tb8)

In [245]:
#mean_wind_tb10= block_mean(indep_wind_tb10, sbi_start_tb10, impact_start_tb10)

In [246]:
#mean_wind_tb11= block_mean(indep_wind_tb11, sbi_start_tb11, impact_start_tb11)

In [247]:
#mean_wind_tb12= block_mean(indep_wind_tb12, sbi_start_tb12, impact_start_tb12)

In [248]:
#mean_wind_tb14= block_mean(indep_wind_tb14, sbi_start_tb14, impact_start_tb14)

In case we use 1- minute mean environmental data from the beginning, the lines above are not needed. We use the following definition instead:

In [148]:
def environment_mean(data, ref_data):

    mean_environment = []
    for i in range(len(data)):
        for j in range (len(data[i])):
            mean_environment.append(data[i][j])
            
    
    while len(mean_environment) > len(ref_data):
        del mean_environment[- 1]
        mean_environment = mean_environment
    
    return mean_environment

In [149]:
mean_wind_tb4 = environment_mean(indep_wind_tb4, maxima_values_tb4)

In [150]:
mean_wind_tb5 = environment_mean(indep_wind_tb5, maxima_values_tb5)

In [151]:
mean_wind_tb6 = environment_mean(indep_wind_tb6, maxima_values_tb6)

In [152]:
mean_wind_tb8 = environment_mean(indep_wind_tb8, maxima_values_tb8)

In [153]:
mean_wind_tb10 = environment_mean(indep_wind_tb10, maxima_values_tb10)

In [154]:
mean_wind_tb11 = environment_mean(indep_wind_tb11, maxima_values_tb11)

In [155]:
mean_wind_tb12 = environment_mean(indep_wind_tb12, maxima_values_tb12)

In [156]:
mean_wind_tb14 = environment_mean(indep_wind_tb14, maxima_values_tb14)

## Mean block wave height

In [157]:
mean_waveH_tb4 = environment_mean(indep_waveH_tb4, maxima_values_tb4)

In [158]:
mean_waveH_tb5 = environment_mean(indep_waveH_tb5, maxima_values_tb5)

In [159]:
mean_waveH_tb6 = environment_mean(indep_waveH_tb6, maxima_values_tb6)

In [160]:
mean_waveH_tb8 = environment_mean(indep_waveH_tb8, maxima_values_tb8)

In [161]:
mean_waveH_tb10 = environment_mean(indep_waveH_tb10, maxima_values_tb10)

In [162]:
mean_waveH_tb11 = environment_mean(indep_waveH_tb11, maxima_values_tb11)

In [163]:
mean_waveH_tb12 = environment_mean(indep_waveH_tb12, maxima_values_tb12)

In [164]:
mean_waveH_tb14 = environment_mean(indep_waveH_tb14, maxima_values_tb14)

## Mean block wave peak period

In [165]:
mean_waveTp_tb4 = environment_mean(indep_waveTp_tb4, maxima_values_tb4)

In [166]:
mean_waveTp_tb5 = environment_mean(indep_waveTp_tb5, maxima_values_tb5)

In [167]:
mean_waveTp_tb6 = environment_mean(indep_waveTp_tb6, maxima_values_tb6)

In [168]:
mean_waveTp_tb8 = environment_mean(indep_waveTp_tb8, maxima_values_tb8)

In [169]:
mean_waveTp_tb10 = environment_mean(indep_waveTp_tb10, maxima_values_tb10)

In [170]:
mean_waveTp_tb11 = environment_mean(indep_waveTp_tb11, maxima_values_tb11)

In [171]:
mean_waveTp_tb12 = environment_mean(indep_waveTp_tb12, maxima_values_tb12)

In [172]:
mean_waveTp_tb14 = environment_mean(indep_waveTp_tb14, maxima_values_tb14)

## Mean block wave zero up-crossing period

In [173]:
mean_waveTz_tb4 = environment_mean(indep_waveTz_tb4, maxima_values_tb4)

In [174]:
mean_waveTz_tb5 = environment_mean(indep_waveTz_tb5, maxima_values_tb5)

In [175]:
mean_waveTz_tb6 = environment_mean(indep_waveTz_tb6, maxima_values_tb6)

In [176]:
mean_waveTz_tb8 = environment_mean(indep_waveTz_tb8, maxima_values_tb8)

In [186]:
mean_waveTz_tb10 = environment_mean(indep_waveTz_tb10, maxima_values_tb10)

In [177]:
mean_waveTz_tb11 = environment_mean(indep_waveTz_tb11, maxima_values_tb11)

In [178]:
mean_waveTz_tb12 = environment_mean(indep_waveTz_tb12, maxima_values_tb12)

In [179]:
mean_waveTz_tb14 = environment_mean(indep_waveTz_tb14, maxima_values_tb14)

# Response Emulator

In [180]:
maxima_deflection= maxima_values_tb4+ maxima_values_tb5+ maxima_values_tb6+ maxima_values_tb8+ maxima_values_tb10+ maxima_values_tb11+ maxima_values_tb12+ maxima_values_tb14

In [181]:
maxima_deflection_tower= maxima_val_tower_tb4 + maxima_val_tower_tb5+ maxima_val_tower_tb6 + maxima_val_tower_tb8 + maxima_val_tower_tb10 +maxima_val_tower_tb11 + maxima_val_tower_tb12+ maxima_val_tower_tb14

In [182]:
mean_wind= mean_wind_tb4 + mean_wind_tb5+ mean_wind_tb6+ mean_wind_tb8+ mean_wind_tb10+ mean_wind_tb11+ mean_wind_tb12+ mean_wind_tb14

In [183]:
mean_waveH= mean_waveH_tb4 + mean_waveH_tb5+ mean_waveH_tb6+ mean_waveH_tb8+ mean_waveH_tb10+ mean_waveH_tb11+ mean_waveH_tb12+ mean_waveH_tb14

In [184]:
mean_waveTp= mean_waveTp_tb4 + mean_waveTp_tb5+ mean_waveTp_tb6+ mean_waveTp_tb8+ mean_waveTp_tb10+ mean_waveTp_tb11+ mean_waveTp_tb12+ mean_waveTp_tb14

In [187]:
mean_waveTz= mean_waveTz_tb4 + mean_waveTz_tb5+ mean_waveTz_tb6+ mean_waveTz_tb8+ mean_waveTz_tb10+ mean_waveTz_tb11+ mean_waveTz_tb12+ mean_waveTz_tb14

In [188]:
print(len(maxima_deflection))
print(len(maxima_deflection_tower))
print(len(mean_wind))
print(len(mean_waveH))
print(len(mean_waveTp))
print(len(mean_waveTz))

1272
1272
1272
1272
1272
1272


In [189]:
fig, axs = plt.subplots(2, )
axs[0, 0].plot(sbitroot_installation_tb4.deflection)
axs[0, 0].set_title('TB4')
axs[0, 1].plot(sbitroot_installation_tb5.deflection)
axs[0, 1].set_title('TB5')
axs[1, 0].plot(sbitroot_installation_tb6.deflection)
axs[1, 0].set_title('TB6')
axs[1, 1].plot(sbitroot_installation_tb8.deflection)
axs[1, 1].set_title('TB8')

for ax in axs.flat:
    ax.set(xlabel='x-label', ylabel='deflection')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

<IPython.core.display.Javascript object>

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## Data

In [190]:
data_unsorted = {'windSpeed':mean_wind,'waveHeight':mean_waveH, 'waveTp':mean_waveTp,'waveTz':mean_waveTz,'peakDeflection':maxima_deflection, 'peakDeflectionTower':maxima_deflection_tower}
data_unsorted = pd.DataFrame(data_unsorted)
data_unsorted["peakDeflection"] = 100 * data_unsorted["peakDeflection"] # convert deflection into cm
data_unsorted["peakDeflectionTower"] = 100 * data_unsorted["peakDeflectionTower"]
data_unsorted.to_csv('/Volumes/MASTERTHESE/MasterThese/data_unsorted.csv')

In [190]:
#mean_wind.sort()
#maxima_deflection.sort()

#data_sorted = {'windspeed':mean_wind,'deflection':maxima_deflection}
#data_sorted = pd.DataFrame(data_sorted)
#data_sorted["deflection"] = 100 * data_sorted["deflection"] # convert deflection into cm
#data_sorted.to_csv('/Volumes/MASTERTHESE/MasterThese/data_sorted.csv')